**Post Lab-1**

**Post Lab-2**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import convolve, correlate

# Normalize audio data
def normalize_audio(audio_data):
    audio_data = audio_data / np.max(np.abs(audio_data))
    return audio_data

# Linear and circular convolution
def perform_convolution(audio_file, impulse_response_file):
    fs_audio, audio = wavfile.read(audio_file)
    fs_ir, impulse_response = wavfile.read(impulse_response_file)

    # Convert to mono if stereo
    if audio.ndim > 1:
        audio = audio[:, 0]
    if impulse_response.ndim > 1:
        impulse_response = impulse_response[:, 0]

    audio = normalize_audio(audio)
    impulse_response = normalize_audio(impulse_response)

    # Linear convolution
    linear_conv = convolve(audio, impulse_response, mode='full')

    # Circular convolution using FFT
    N = max(len(audio), len(impulse_response))
    circular_conv = np.fft.ifft(np.fft.fft(audio, N) * np.fft.fft(impulse_response, N)).real

    # Visualization
    plt.figure(figsize=(12, 6))
    plt.subplot(2, 1, 1)
    plt.title('Linear Convolution')
    plt.plot(linear_conv, color='blue')
    plt.subplot(2, 1, 2)
    plt.title('Circular Convolution')
    plt.plot(circular_conv, color='red')
    plt.tight_layout()
    plt.show()

    return linear_conv, circular_conv

In [ ]:
# Cross-correlation and autocorrelation
def perform_correlations(clean_file, noisy_file, periodic_file):
    files = {
        'Clean': clean_file,
        'Noisy': noisy_file,
        'Periodic': periodic_file
    }

    results = {}
    plt.figure(figsize=(15, 10))

    for i, (name, file) in enumerate(files.items()):
        fs, audio = wavfile.read(file)
        if audio.ndim > 1:
            audio = audio[:, 0]
        audio = normalize_audio(audio)

        # Autocorrelation
        autocorr = correlate(audio, audio, mode='full')
        autocorr = autocorr / np.max(autocorr)

        # Cross-correlation (with clean reference)
        if name != 'Clean':
            fs_c, clean_audio = wavfile.read(clean_file)
            if clean_audio.ndim > 1:
                clean_audio = clean_audio[:, 0]
            clean_audio = normalize_audio(clean_audio)
            crosscorr = correlate(audio, clean_audio, mode='full')
            crosscorr = crosscorr / np.max(crosscorr)
        else:
            crosscorr = autocorr

        results[name] = (autocorr, crosscorr)

        # Visualization
        plt.subplot(len(files), 2, 2*i + 1)
        plt.title(f'{name} Audio - Autocorrelation')
        plt.plot(autocorr, color='green')

        plt.subplot(len(files), 2, 2*i + 2)
        plt.title(f'{name} Audio - Cross-correlation')
        plt.plot(crosscorr, color='purple')

    plt.tight_layout()
    plt.show()

    return results